Reference taken from https://github.com/samoturk/mol2vec/blob/master/examples/02%20-%20Supervised%20machine%20learning%20with%20Mol2vec.ipynb

In [1]:
!pip install git+https://github.com/samoturk/mol2vec
    

  Cloning https://github.com/samoturk/mol2vec to /private/var/folders/3p/mwxtj1wj179dmn25cxrq38380000gn/T/pip-req-build-ed1r2b7f
  Running command git clone -q https://github.com/samoturk/mol2vec /private/var/folders/3p/mwxtj1wj179dmn25cxrq38380000gn/T/pip-req-build-ed1r2b7f
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14026 sha256=902e226948cc2e52eb4e133b709825aebcfa0609628630978e12722667bd5af1
  Stored in directory: /private/var/folders/3p/mwxtj1wj179dmn25cxrq38380000gn/T/pip-ephem-wheel-cache-36bl0mc2/wheels/28/66/2a/543e91280db70c489a72de9c9fa26dca4caa7113721fb03b86
Successfully built mol2vec


In [2]:
import numpy as np
import pandas as pd
import sys, os

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))+'/mol2vec')

from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

RDKit WARNING: [02:43:58] Enabling RDKit 2019.09.3 jupyter extensions
/Users/shrutidudwadkar/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Importing the BindingDB_Kd dataset

In [3]:
from tdc.multi_pred import DTI
bindingDB_data = DTI(name = 'BindingDB_Kd')

Found local copy...
Loading...
Done!


In [4]:
bindingDB_cold_split = bindingDB_data.get_split(method = 'cold_split', column_name = 'Drug', seed = 42, frac = [0.7, 0.1, 0.2])


In [5]:
train_set = bindingDB_cold_split['train']

In [6]:
train_set

,Drug_ID,Drug,Target_ID,Target,Y
0,4316.0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.490
1,4293.0,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.830
2,1612.0,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.160
3,4369102.0,CCN[C@H]1CN(CCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.320
4,68844.0,CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.130
...,...,...,...,...,...
34696,76311094.0,O=C(NCCCCS(=O)(=O)c1ccccc1)c1ccc2nccn2c1,P43490,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,860.000
34697,57406853.0,CCOc1cc(C(C)(C)C)ccc1C1=N[C@@](C)(c2ccc(Cl)cc2...,NaN,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,2.900
34698,58573469.0,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1C(=O)[C@@](C)(CC...,NaN,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,0.045
34699,113557.0,CCCCCCCOC1OC(CO)C(O)C(O)C1O,P08191,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,17.000


In [7]:
dataSource = train_set.drop(['Target_ID', 'Drug_ID', 'Y'], axis=1)
dataSource['Mol'] = ''


Load the pre-trained Mol2vec model 

In [8]:
model = word2vec.Word2Vec.load('model_300dim.pkl')


In [9]:
from rdkit import Chem
import tensorflow as tf
import torch

for ind, drug in enumerate(dataSource['Drug']):
    mol = Chem.MolFromSmiles(drug)
    dataSource['Mol'][ind] = mol
    
    

In [10]:
dataSource['Sentence'] = dataSource.apply(lambda x: MolSentence(mol2alt_sentence(x['Mol'], 1)), axis=1)


Method taken from mol2vec and updated as per the changes in the gensim.
Tweeked the 'sentences2vec' method provided in Mol2Vec library to make it compatible with gensim 4.0

In [11]:
def sentences2vec(sentences, model, unseen=None):
    
    #Change made in this line as vocab is removed in gensim 4.0
    #keys = set(model.wv.vocab.keys())
    keys = set(model.wv.key_to_index.keys())
    
    vec = []
    if unseen:
        unseen_vec = model.wv.word_vec(unseen)

    for sentence in sentences:
        if unseen:
            vec.append(sum([model.wv.word_vec(y) if y in set(sentence) & keys
                       else unseen_vec for y in sentence]))
        else:
            vec.append(sum([model.wv.word_vec(y) for y in sentence 
                            if y in set(sentence) & keys]))
    return np.array(vec)

In [12]:
dataSource['mol2vec'] = ''
dataSource['mol2vec'] = [DfVec(x) for x in sentences2vec(dataSource['Sentence'], model, unseen='UNK')]


In [13]:
X = np.array([x.vec for x in dataSource['mol2vec']])

In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=X, shuffle=True, batch_size=256)

In [15]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(linkage='average', n_clusters=50)

clusters=[]
for i, batch in enumerate(train_loader):
    clustering.fit(batch)
    clusters.extend(clustering.labels_)

Grouping the dataframe based on the clusters formed

In [16]:
dataSource['Clusters'] = clusters
by_clusters = dataSource.groupby(['Clusters'])

In [17]:
dataSource

,Drug,Target,Mol,Sentence,mol2vec,Clusters
0,COc1ccc(CNS(=O)(=O)c2ccc(S(N)(=O)=O)s2)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3975275337, 864674487, 2076190208...","(300,) dimensional vector",22
1,NS(=O)(=O)c1ccc(S(=O)(=O)NCc2cccs2)s1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"<img data-content=""rdkit/molecule"" src=""data:i...","(847957139, 3275683399, 999334238, 3532053313,...","(300,) dimensional vector",5
2,COc1ccc(N2CC(O)c3cc(S(N)(=O)=O)sc3S2(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3975275337, 864674487, 2076190208...","(300,) dimensional vector",4
3,CCN[C@H]1CN(CCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3542456614, 2245384272, 773607102...","(300,) dimensional vector",5
4,CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3542456614, 2245384272, 773607102...","(300,) dimensional vector",3
...,...,...,...,...,...,...
34696,O=C(NCCCCS(=O)(=O)c1ccccc1)c1ccc2nccn2c1,MNPAAEAEFNILLATDSYKVTHYKQYPPNTSKVYSYFECREKKTEN...,"<img data-content=""rdkit/molecule"" src=""data:i...","(864942730, 1510328189, 2246699815, 1054767590...","(300,) dimensional vector",6
34697,CCOc1cc(C(C)(C)C)ccc1C1=N[C@@](C)(c2ccc(Cl)cc2...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3542456614, 2245384272, 399408866...","(300,) dimensional vector",17
34698,CC(C)[C@@H](CS(=O)(=O)C(C)C)N1C(=O)[C@@](C)(CC...,MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKD...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3537119515, 2245273601, 300433380...","(300,) dimensional vector",15
34699,CCCCCCCOC1OC(CO)C(O)C(O)C1O,MKRVITLFAVLLMGWSVNAWSFACKTANGTAIPIGGGSANVYVNLA...,"<img data-content=""rdkit/molecule"" src=""data:i...","(2246728737, 3542456614, 2245384272, 117312591...","(300,) dimensional vector",30


In [18]:
for cluster, data in by_clusters:
    print(f"First 2 entries for cluster: ", cluster)
    print("------------------------")
    print(data.head(2))

First 2 entries for cluster:  0
------------------------
                                                Drug  \
10           NS(=O)(=O)c1ccc(C(=O)NCc2cc(F)ccc2F)cc1   
14  NS(=O)(=O)c1ccc(C(=O)NCc2c(F)c(F)c(F)c(F)c2F)cc1   

                                               Target  \
10  SHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPL...   
14  SHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPL...   

                                                  Mol  \
10  <img data-content="rdkit/molecule" src="data:i...   
14  <img data-content="rdkit/molecule" src="data:i...   

                                             Sentence  \
10  (847957139, 3275683399, 999334238, 3532053313,...   
14  (847957139, 3275683399, 999334238, 3532053313,...   

                      mol2vec  Clusters  
10  (300,) dimensional vector         0  
14  (300,) dimensional vector         0  
First 2 entries for cluster:  1
------------------------
                                           Drug  \
159  O=C1c2cc(F)ccc2-c2c

                                                 Drug  \
34  C[C@H](N)C(=O)Nc1nc(-c2ccc(Cc3ccccc3)cc2)c(-c2...   
58  Cc1ccc(-n2nc(C(C)(C)C)cc2NC(=O)Nc2ccc(OCCN3CCO...   

                                               Target  \
34  MSDAVSSDRNFPNSTNLPRNPSMADYEARIFTFGTWIYSVNKEQLA...   
58  HHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKH...   

                                                  Mol  \
34  <img data-content="rdkit/molecule" src="data:i...   
58  <img data-content="rdkit/molecule" src="data:i...   

                                             Sentence  \
34  (2246728737, 3537119515, 2245273601, 265540621...   
58  (2246728737, 422715066, 3217380708, 3207567135...   

                      mol2vec  Clusters  
34  (300,) dimensional vector        19  
58  (300,) dimensional vector        19  
First 2 entries for cluster:  20
------------------------
                                                  Drug  \
75   COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...   
210       CC

398  (300,) dimensional vector        40  
First 2 entries for cluster:  41
------------------------
                                                  Drug  \
124  CO[C@@H]1O[C@H](CO)[C@@H](O[C@@H]2O[C@H](CO)[C...   
397  CCN1CCN(Cc2ccc(NC(=O)Nc3ccc(Oc4cc(NC)ncn4)cc3)...   

                                                Target  \
124  MSNVPHKSSLPEGIRPGTVLRIRGLVPPNASRFHVNLLCGEEQGSD...   
397  MDYDFKAKLAAERERVEDLFEYEGCKVGRGTYGHVYKARRKDGKDE...   

                                                   Mol  \
124  <img data-content="rdkit/molecule" src="data:i...   
397  <img data-content="rdkit/molecule" src="data:i...   

                                              Sentence  \
124  (2246728737, 3975275337, 864674487, 1978844073...   
397  (2246728737, 3542456614, 2245384272, 225184566...   

                       mol2vec  Clusters  
124  (300,) dimensional vector        41  
397  (300,) dimensional vector        41  
First 2 entries for cluster:  42
------------------------
            